# Using SDBN Click Model To Overcome Position Bias

This section we use the _Simplified Dynamic Bayesian Network_ (SDBN) to overcome the position bias that we saw with direct Click-Through-Rate. We consider the SDBN judgments and how they compare to just the click through rate.

In [1]:
import sys
sys.path.append('..')
from ltr.sdbn_functions import all_sessions, get_sessions
from aips import fetch_products, render_judged
import matplotlib.pyplot as plt
import numpy
import pandas 
#from ltr.sdbn_functions import *
import glob 

# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
sessions = all_sessions()
products = fetch_products(doc_ids=sessions['doc_id'].unique())
products

,image,upc,name,name_ngram,name_omit_norms,name_txt_en_split,manufacturer,long_description,short_description,id,_version_
0,"<img height=""100"" src=""../data/retrotech/image...",885909471812,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 16GB -...,Apple&#xAE;,The all-new thinner and lighter design makes i...,"9.7"" widescreen display; 802.11a/b/g/n Wi-Fi; ...",04f56274-f37a-4ffa-8e81-a84329594a05,1798424868871471105
1,"<img height=""100"" src=""../data/retrotech/image...",885909472376,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 32GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 32GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 32GB -...,Apple&#xAE; - iPad&#xAE; 2 with Wi-Fi - 32GB -...,Apple&#xAE;,The all-new thinner and lighter design makes i...,"9.7"" widescreen display; 802.11a/b/g/n Wi-Fi; ...",16cffa8c-dd48-42e7-863b-2229f843a55b,1798424868877762560
2,"<img height=""100"" src=""../data/retrotech/image...",882777064207,"Westinghouse - 22"" Class - lCD - 720p - 60Hz -...","Westinghouse - 22"" Class - lCD - 720p - 60Hz -...","Westinghouse - 22"" Class - lCD - 720p - 60Hz -...","Westinghouse - 22"" Class - lCD - 720p - 60Hz -...",Westinghouse,This LCD HDTV supports high-definition signals...,ENERGY STAR Qualified,8506b200-f71b-4ccb-a455-d2cc15640840,1798424868940677122
3,"<img height=""100"" src=""../data/retrotech/image...",885909394494,Apple&#xAE; - iPhone 4 with 16GB Memory - Whit...,Apple&#xAE; - iPhone 4 with 16GB Memory - Whit...,Apple&#xAE; - iPhone 4 with 16GB Memory - Whit...,Apple&#xAE; - iPhone 4 with 16GB Memory - Whit...,Apple&#xAE;,"This slim, powerful iPhone features a high-qua...","iPhone iOS 4 operating systemWi-Fi3.5"" Retina ...",09b65e46-e7d7-4315-a2d6-9d91e7002275,1798424869040291840
4,"<img height=""100"" src=""../data/retrotech/image...",885170038875,"Panasonic - 9"" Widescreen Portable DVD Player","Panasonic - 9"" Widescreen Portable DVD Player","Panasonic - 9"" Widescreen Portable DVD Player","Panasonic - 9"" Widescreen Portable DVD Player",Panasonic,Watch your favorite movies on the go with this...,"Plays DVD, DVD+R DL, DVD-R DL, DVD+R/RW, DVD-R...",b589cbf9-4ddb-4f09-b48a-ca89a6df28d5,1798424869702991874
...,...,...,...,...,...,...,...,...,...,...,...
306,"<img height=""100"" src=""../data/retrotech/image...",24543742180,Star Wars: The Complete Saga [9 Discs / Blu-ra...,Star Wars: The Complete Saga [9 Discs / Blu-ra...,Star Wars: The Complete Saga [9 Discs / Blu-ra...,Star Wars: The Complete Saga [9 Discs / Blu-ra...,\N,,,202f29f2-158d-49eb-b97b-1bb5cc51b3d9,1798424876875251715
307,"<img height=""100"" src=""../data/retrotech/image...",786936805017,Disney's A Christmas Carol - Blu-ray Disc,Disney's A Christmas Carol - Blu-ray Disc,Disney's A Christmas Carol - Blu-ray Disc,Disney's A Christmas Carol - Blu-ray Disc,\N,,,4edb5abc-7603-4b20-a44a-4b13d4ce187e,1798424877363888131
308,"<img height=""100"" src=""../data/retrotech/image...",786936817232,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,\N,,,e3c1706e-64d9-4f89-bf3b-f3227d8cb179,1798424877464551425
309,"<img height=""100"" src=""../data/retrotech/image...",786936817218,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,Pirates of the Caribbean: On Stranger Tides - ...,\N,,,8bc67681-ece2-4315-9222-8ac958aa7b04,1798424877464551432


# Listing 11.7

Click models overcome position bias by learning an examine probability on each ranking. SDBN tracks examines relative to the the last click. This code marks last click position per session so we can compute examine probabilities.

In [3]:
#%load -s calculate_examine_probability ../ltr/sdbn_functions.py
def calculate_examine_probability(sessions):
    last_click_per_session = sessions.groupby(
        ["clicked", "sess_id"])["rank"].max()[True]
    sessions["last_click_rank"] = last_click_per_session
    sessions["examined"] = \
      sessions["rank"] <= sessions["last_click_rank"]
    return sessions

In [4]:
sessions = get_sessions("dryer")
calculate_examine_probability(sessions)

,query,rank,doc_id,clicked,last_click_rank,examined
sess_id,,,,,,
2,dryer,0.0,12505451713,False,12.0,True
2,dryer,1.0,84691226727,True,12.0,True
2,dryer,2.0,883049066905,False,12.0,True
2,dryer,3.0,14381196320,False,12.0,True
2,dryer,4.0,74108056764,False,12.0,True
...,...,...,...,...,...,...
5001,dryer,15.0,36725561977,True,15.0,True
5001,dryer,16.0,36725578241,False,15.0,False
5001,dryer,17.0,12505527456,False,15.0,False


# Listing 11.8

Aggregate clicks and examine counts

In [5]:
#%load -s calculate_clicked_examined ../ltr/sdbn_functions.py
def calculate_clicked_examined(sessions):
    sessions = calculate_examine_probability(sessions)
    return sessions[sessions["examined"]] \
        .groupby("doc_id")[["clicked", "examined"]].sum()

In [6]:
sessions = get_sessions("dryer")
calculate_clicked_examined(sessions)

,clicked,examined
doc_id,,
12505451713,355,2707
12505525766,268,974
12505527456,110,428
14381196320,217,1202
36172950027,97,971
36725561977,119,572
36725578241,130,477
48231011396,166,423
48231011402,213,818


# Listing 11.9

We compute a grade - a probability of relevance - by dividing the clicks by examines. This is the kind of dynamic 'click thru rate' of SDBN, that accounts for whether the result was actually seen by users, not just whether it was shown on the screen.

In [7]:
#%load -s calculate_grade ../ltr/sdbn_functions.py
def calculate_grade(sessions):
    sessions = calculate_clicked_examined(sessions)
    sessions["grade"] = sessions["clicked"] / sessions["examined"]
    return sessions.sort_values("grade", ascending=False)

In [8]:
query = "dryer"
sessions = get_sessions(query)
calculate_grade(sessions)

,clicked,examined,grade
doc_id,,,
856751002097,133,323,0.411765
48231011396,166,423,0.392435
84691226727,804,2541,0.316411
74108007469,208,708,0.293785
12505525766,268,974,0.275154
36725578241,130,477,0.272537
48231011402,213,818,0.260391
12505527456,110,428,0.257009
74108096487,235,1097,0.214221


# Figure 11.6 Source Code

In [9]:
render_judged(products, calculate_grade(sessions), grade_col="grade", label=f"SDBN judgments for q={query}")

,grade,image,upc,name,short_description
0,0.411765,,856751002097,Practecol - Dryer Balls (2-Pack),"Suitable for use on most dry cycles; reduces lint, static and wrinkles; improves heat circulation; 2-pack"
1,0.392435,,48231011396,LG - 3.5 Cu. Ft. 7-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedDigital controls; 7 cycles; SpeedWash cycle; 9 wash options; delay-wash; SenseClean system; 6Motion technology; TrueBalance antivibration system
2,0.316411,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
3,0.293785,,74108007469,Conair - 1875-Watt Folding Handle Hair Dryer - Blue,2 heat/speed settings; cool shot button; dual voltage; professional-length line cord
4,0.275154,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
5,0.272537,,36725578241,Samsung - 7.3 Cu. Ft. 7-Cycle Electric Dryer - White,Soft-touch dial controls; 7 preset drying cycles; 4 temperature settings; powdercoat drum; noise reduction package
6,0.260391,,48231011402,LG - 7.1 Cu. Ft. 7-Cycle Electric Dryer - White,Electronic controls with LED display; 7 cycles; Dial-A-Cycle option; sensor dry system; 5 temperature levels; 5 drying levels; NeveRust drum; LoDecibel quiet operation
7,0.257009,,12505527456,"Smart Choice - 1/2"" Safety+PLUS Stainless-Steel Gas Dryer Connector","Safety+PLUS automatic shut-off valve; leak detection solution; pipe thread sealant; 60,500 BTU; CSA approved"
8,0.214221,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
9,0.208042,,36725561977,Samsung - 3.5 Cu. Ft. 6-Cycle High-Efficiency Washer - White,ENERGY STAR QualifiedSoft dial touch pad controls; 6 cycles; delay-start; child lock; Vibration Reduction Technology


# Listing 11.10 Source Code

In [10]:
query = "transformers dark of the moon"
sessions = get_sessions(query)
calculate_grade(sessions)

,clicked,examined,grade
doc_id,,,
97360810042,412,642,0.641745
400192926087,62,129,0.480620
97363560449,96,243,0.395062
97363532149,42,130,0.323077
93624956037,41,154,0.266234
47875842328,367,1531,0.239713
47875841420,217,960,0.226042
25192107191,176,1082,0.162662
786936817218,118,777,0.151866


# Figure 11.7 Source Code

In [11]:
render_judged(products, calculate_grade(sessions), grade_col="grade", label=f"SDBN judgments for q={query}")

,grade,image,upc,name,short_description
0,0.641745,,97360810042,Transformers: Dark of the Moon - Blu-ray Disc,
1,0.480620,,400192926087,Transformers: Dark of the Moon - Original Soundtrack - CD,
2,0.395062,,97363560449,Transformers: Dark of the Moon - Widescreen Dubbed Subtitle - DVD,
3,0.323077,,97363532149,Transformers: Revenge of the Fallen - Widescreen Dubbed Subtitle - DVD,
4,0.266234,,93624956037,Transformers: Dark of the Moon - Original Soundtrack - CD,
5,0.239713,,47875842328,Transformers: Dark of the Moon Stealth Force Edition - Nintendo Wii,Transform into an epic hero or a vehicular villain
6,0.226042,,47875841420,Transformers: Dark of the Moon Decepticons - Nintendo DS,Transform into an epic hero or a vehicular villain
7,0.162662,,25192107191,Fast Five - Widescreen - Blu-ray Disc,
8,0.151866,,786936817218,Pirates Of The Caribbean: On Stranger Tides (3-D) - Blu-ray 3D,
9,0.151866,,786936817218,Pirates of the Caribbean: On Stranger Tides - Blu-ray 3D,


Up next: [Dealing with Low Confidence Situations](3.SDBN-Confidence-Bias.ipynb)